# LunAPI : projects

Links to notebooks in this repository: [Index](./00_overview.ipynb) | [Individuals](./01_indivs.ipynb) | [Projects](./02_projects.ipynb) | [Staging](./03_staging.ipynb) | [Models](./04_models.ipynb) | [Advanced](./98_advanced.ipynb) | [Reference](./99_reference.ipynb)

We can then attach a sample list, either from a file (`read_sample_list()`) or build one on-the-fly.  This function returns the number of observations (EDF/annotation pairs), but stores the information on the samples internally.

In [3]:
proj.build( [ 'tutorial/' , '-ext=-profusion.xml' ] ) 

3

You can see the information of the attached sample list as follows:

In [4]:
proj.sample_list()

[('learn-nsrr01',
  'tutorial/edfs/learn-nsrr01.edf',
  {'tutorial/edfs/learn-nsrr01-profusion.xml'}),
 ('learn-nsrr02',
  'tutorial/edfs/learn-nsrr02.edf',
  {'tutorial/edfs/learn-nsrr02-profusion.xml'}),
 ('learn-nsrr03',
  'tutorial/edfs/learn-nsrr03.edf',
  {'tutorial/edfs/learn-nsrr03-profusion.xml'})]

To apply a Luna command to all individuals in a sample list use `eval()`:

In [5]:
res = proj.eval( 'HEADERS' ) 

In [6]:
res.keys()

dict_keys(['HEADERS: BL', 'HEADERS: CH'])

In [7]:
lp.show( res ) 

HEADERS: BL


,ID,EDF_ID,EDF_TYPE,NR,NS,NS_ALL,REC_DUR,REC_DUR_HMS,REC_DUR_SEC,START_DATE,START_TIME,STOP_TIME,TOT_DUR_HMS,TOT_DUR_SEC
0,learn-nsrr01,.,EDF,40920,14,14,1.0,11:22:00,40920.0,01.01.85,21.58.17,09.20.17,11.22.00,40920.0
1,learn-nsrr02,.,EDF,35850,14,14,1.0,09:57:30,35850.0,01.01.85,21.18.06,07.15.36,09.57.30,35850.0
2,learn-nsrr03,.,EDF,40920,14,14,1.0,11:22:00,40920.0,01.01.85,20.15.00,07.37.00,11.22.00,40920.0


HEADERS: CH


,ID,CH,DMAX,DMIN,PDIM,PMAX,PMIN,POS,SENS,SR,TRANS,TYPE
0,learn-nsrr01,ABDO_RES,127,-128,.,-1.0,1.0,11,-0.007843,10.0,.,EFFORT
1,learn-nsrr02,ABDO_RES,127,-128,.,-1.0,1.0,11,-0.007843,10.0,.,EFFORT
2,learn-nsrr03,ABDO_RES,127,-128,.,-1.0,1.0,11,-0.007843,10.0,.,EFFORT
3,learn-nsrr01,AIRFLOW,127,-128,.,-1.0,1.0,9,-0.007843,10.0,.,AIRFLOW
4,learn-nsrr02,AIRFLOW,127,-128,.,-1.0,1.0,9,-0.007843,10.0,.,AIRFLOW
5,learn-nsrr03,AIRFLOW,127,-128,.,-1.0,1.0,9,-0.007843,10.0,.,AIRFLOW
6,learn-nsrr01,ECG,127,-128,mV,1.25,-1.25,4,0.009804,250.0,.,ECG
7,learn-nsrr02,ECG,127,-128,mV,1.25,-1.25,4,0.009804,250.0,.,ECG
8,learn-nsrr03,ECG,127,-128,mV,1.25,-1.25,4,0.009804,250.0,.,ECG
9,learn-nsrr01,EEG,127,-128,uV,125.0,-125.0,8,0.980392,125.0,.,EEG


### Working with projects

A _project_ is defined by a list of individual samples (EDFs/annotations).  We can then attach a sample list, either from a file (`read_sample_list()`) or build one on-the-fly.  This function returns the number of observations (EDF/annotation pairs), which is 3 in this case, but stores the information on the samples internally.

In [ ]:
proj.build( [ 'tutorial/' , '-ext=-profusion.xml' ] ) 

You can see the information of the attached sample list with the `sample_list()` function (without any arguments):

In [ ]:
proj.sample_list()

### Reading a sample-list from a file

If you have a pre-existing sample list (e.g. manually created, or generated by Luna's `--build` option), you can alternatively attach it with `sample_list( <filename> )`:

In [ ]:
proj.sample_list( 'tutorial/s.lst' )

In [ ]:
proj.sample_list()

Note in this instance the pre-existing sample list uses _relative_ path encoding, i.e. with the assumption that Luna would be run from the `tutorial/` folder directly.  We can see the original file:

In [ ]:
%%sh
cat tutorial/s.lst

When building the sample-list lunapi correctly specified the correct paths (i.e. here `tutorial/edfs/` etc rather than just `edfs`).  To append an additonal path (to all files in a sample list), one can first set the Luna variable `path` - which needs to be done _before_ running `sample_list()`:

In [ ]:
proj.opt( 'path' , 'tutorial/' )

In [ ]:
proj.sample_list('tutorial/s.lst')

In [ ]:
proj.sample_list()

This is similar to invoking Luna on the command line (if from this directory) as follows:

In [ ]:
import lunapi as lp
proj = lp.proj()

In [2]:
proj.build( [ 'tutorial/' , '-ext=-profusion.xml' ] )
proj.sample_list()

[('learn-nsrr01',
  'tutorial//edfs/learn-nsrr01.edf',
  {'tutorial//edfs/learn-nsrr01-profusion.xml'}),
 ('learn-nsrr02',
  'tutorial//edfs/learn-nsrr02.edf',
  {'tutorial//edfs/learn-nsrr02-profusion.xml'}),
 ('learn-nsrr03',
  'tutorial//edfs/learn-nsrr03.edf',
  {'tutorial//edfs/learn-nsrr03-profusion.xml'})]

In [25]:
lp.resources.POPS_PATH = '/Users/smp37/dropbox/projects/moonlight/pops/'
proj.pops('EEG')

___________________________________________________________________
Processing: learn-nsrr01 | tutorial//edfs/learn-nsrr01.edf
 duration 11.22.00, 40920s | time 21.58.17 - 09.20.17 | date 01.01.85

 signals: 14 (of 14) selected in a standard EDF file
  SaO2 | PR | EEG_sec | ECG | EMG | EOG_L | EOG_R | EEG
  AIRFLOW | THOR_RES | ABDO_RES | POSITION | LIGHT | OX_STAT
 ..................................................................
 CMD #1: COPY
   options: sig=EEG tag=FLT
 copying EEG to EEG_FLT
 ..................................................................
 CMD #2: FILTER
   options: bandpass=0.3,35 ripple=0.01 sig=EEG_FLT tw=0.2
  filtering channel(s): EEG_FLT
 ..................................................................
 CMD #3: COPY
   options: sig=EEG_FLT tag=NORM
 copying EEG_FLT to EEG_FLT_NORM
 ..................................................................
 CMD #4: ROBUST-NORM
   options: epoch sig=EEG_FLT_NORM winsor=0.002
  set epochs to default 30 seconds, 13

,PP_N1,PP_N2,PP_N3,PP_R,PP_W
0,None,None,None,None,None
1,0.000609,0.000331,0.000067,0.000108,0.998885
2,0.001253,0.004371,0.000502,0.000576,0.993297
3,None,None,None,None,None
4,0.000571,0.000295,0.000062,0.000096,0.998975
...,...,...,...,...,...
3918,None,None,None,None,None
3919,None,None,None,None,None
3920,None,None,None,None,None
3921,None,None,None,None,None


efault 30 seconds, 1364 epochs
  iterating over epochs
  robust standardization of 1 signals, winsorizing at 0.002
 ..................................................................
 CMD #5: EDGER
   options: cache=ec1 sig=EEG_FLT
  skipping... cache ec1 not found for this individual...
  set 0 leading/trailing sleep epochs to '?' (given end-wake=120 and end-sleep=5)
  anchoring on sleep epochs only for normative ranges, using 448 of 1364 epochs
  for EEG_FLT, flagged 479 epochs (H1=464, H3=326)
  H(1) bounds: 2.6592 .. 6.75024
  H(3) bounds: 0.492385 .. 1.08752
  setting cache ec1 to store times
  lights-on=03.55.30 (skipping 442 epochs from end)
 ..................................................................
 CMD #6: POPS
   options: alias=CEN,ZEN|EEG_FLT,EEG_FLT_NORM cache=ec1 lib=s2 path=/Users/smp37/dropbox/projects/moonlight/pops/ sig=*
  read 65 valid feature mean/SD ranges from /Users/smp37/dropbox/projects/moonlight/pops//s2.ranges
  from TRIM cache, setting lights_on = 2

In [26]:
proj.strata()

,Command,Stata
0,EDGER,BL
1,EDGER,CH
2,POPS,BL
3,POPS,E
4,POPS,ETYPE
5,POPS,ETYPE_SS
6,POPS,FTR
7,POPS,OBS_PRED
8,POPS,SS


In [27]:
proj.table( 'POPS'  )

,ID,ACC,ACC3,CONF,F1,F13,F1_WGT,K,K3,MCC,...,PREC,PREC3,PREC_WGT,RECALL,RECALL3,RECALL_WGT,REM_LAT_OBS,REM_LAT_PRD,SLP_LAT_OBS,SLP_LAT_PRD
0,learn-nsrr01,0.706667,0.813333,0.715839,0.49383,0.775422,0.693885,0.5588,0.660468,0.564029,...,0.502792,0.768321,0.69898,0.545983,0.788803,0.706667,55.5,123.0,26.5,41.5
1,learn-nsrr02,0.876842,0.924211,0.832154,0.771329,0.899725,0.880526,0.828978,0.857905,0.830441,...,0.75279,0.899157,0.888584,0.817273,0.900615,0.876842,43.5,30.5,45.5,58.5
2,learn-nsrr03,0.669924,0.751357,0.781646,NaN,NaN,NaN,0.454372,0.511217,0.495097,...,0.429245,0.549692,0.767251,NaN,NaN,NaN,None,93.0,56.0,58.0


In [28]:
proj.table( 'POPS' , 'SS' )

,ID,SS,F1,OBS,ORIG,PR1,PREC,PRF,RECALL
0,learn-nsrr01,?,None,157.0,157.0,157.0,None,157.0,None
1,learn-nsrr02,?,None,122.5,122.5,122.5,None,122.5,None
2,learn-nsrr03,?,None,221.5,221.5,221.5,None,221.5,None
3,learn-nsrr01,N1,0.062016,54.5,54.5,10.0,0.2,51.318646,0.036697
4,learn-nsrr02,N1,0.324324,5.5,5.5,13.0,0.230769,24.437311,0.545455
5,learn-nsrr03,N1,0.170213,26.0,26.0,21.0,0.190476,45.513099,0.153846
6,learn-nsrr01,N2,0.82032,261.5,261.5,270.0,0.807407,240.390113,0.833652
7,learn-nsrr02,N2,0.871863,199.0,199.0,179.5,0.91922,173.434682,0.829146
8,learn-nsrr03,N2,0.720682,187.5,187.5,281.5,0.600355,233.486749,0.901333
9,learn-nsrr01,N3,0.129032,8.5,8.5,38.0,0.078947,43.151627,0.352941


In [ ]:
proj.vars()

In [7]:
res = proj.eval( 'STATS sig=EEG' )

___________________________________________________________________
Processing: learn-nsrr01 | tutorial//edfs/learn-nsrr01.edf
 duration 11.22.00, 40920s | time 21.58.17 - 09.20.17 | date 01.01.85

 signals: 14 (of 14) selected in a standard EDF file
  SaO2 | PR | EEG_sec | ECG | EMG | EOG_L | EOG_R | EEG
  AIRFLOW | THOR_RES | ABDO_RES | POSITION | LIGHT | OX_STAT
 ..................................................................
 CMD #1: STATS
   options: sig=EEG
 processing EEG ...
___________________________________________________________________
Processing: learn-nsrr02 | tutorial//edfs/learn-nsrr02.edf
 duration 09.57.30, 35850s | time 21.18.06 - 07.15.36 | date 01.01.85

 signals: 14 (of 14) selected in a standard EDF file
  SaO2 | PR | EEG_sec | ECG | EMG | EOG_L | EOG_R | EEG
  AIRFLOW | THOR_RES | ABDO_RES | POSITION | LIGHT | OX_STAT
 ..................................................................
 CMD #1: STATS
   options: sig=EEG
 processing EEG ...
__________________

In [8]:
lp.show( res )

STATS: CH


,ID,CH,KURT,MAX,MEAN,MIN,P01,P02,P05,P10,...,P60,P70,P80,P90,P95,P98,P99,RMS,SD,SKEW
0,learn-nsrr01,EEG,4.657487,125.0,-0.301199,-124.019608,-124.019608,-124.019608,-70.098039,-24.019608,...,2.45098,4.411765,8.333333,23.039216,72.058824,125.0,125.0,37.801351,37.800154,0.086212
1,learn-nsrr02,EEG,3.546719,125.0,-0.370447,-124.019608,-124.019608,-124.019608,-78.921569,-33.823529,...,4.411765,8.333333,14.215686,30.882353,75.980392,125.0,125.0,41.044234,41.042566,-0.019268
2,learn-nsrr03,EEG,1.345607,125.0,-0.11223,-124.019608,-124.019608,-124.019608,-124.019608,-73.039216,...,3.431373,7.352941,15.196078,70.098039,125.0,125.0,125.0,54.2708,54.270689,-0.011078


In [9]:
proj.strata()

,Command,Stata
0,STATS,CH


In [10]:
df = proj.table( 'STATS' , 'CH' )

In [11]:
df[ ['MEAN','KURT','SKEW' ] ].to_numpy(dtype=float)

array([[-0.3011987 ,  4.65748664,  0.08621198],
       [-0.37044745,  3.54671873, -0.01926794],
       [-0.11222989,  1.34560684, -0.01107757]])

In [12]:
df[ ['ID','KURT'] ] 

,ID,KURT
0,learn-nsrr01,4.657487
1,learn-nsrr02,3.546719
2,learn-nsrr03,1.345607


In [13]:
proj.var( 'vars' , 'ages.txt' )

In [14]:
proj.vars()

{'anterio-frontal': 'AF7,AF3,AFZ,AF8,AF4',
 'anterior': 'FP1,AF7,AF3,F1,F3,F5,F7,FPZ,AFZ,FZ,FP2,AF8,AF4,F2,F4,F6,F8',
 'central': 'FT7,FC5,FC3,FC1,C1,C3,C5,T7,TP7,CP5,CP3,CP1,CPZ,FCZ,CZ,FT8,FC6,FC4,FC2,C2,C4,C6,T8,TP8,CP6,CP4,CP2',
 'centro-parietal': 'CP5,CP3,CP1,CPZ,CP6,CP4,CP2',
 'do_edger': '1',
 'do_reref': '0',
 'frontal': 'F1,F3,F5,F7,FZ,F2,F4,F6,F8',
 'fronto-central': 'FC5,FC3,FC1,FCZ,FC6,FC4,FC2',
 'left': 'FP1,AF7,AF3,F1,F3,F5,F7,FT7,FC5,FC3,FC1,C1,C3,C5,T7,TP7,CP5,CP3,CP1,P1,P3,P5,P7,P9,PO7,PO3,O1',
 'lib': 's2',
 'mid-central': 'C1,C3,C5,CZ,C2,C4,C6',
 'midline': 'IZ,OZ,POZ,PZ,CPZ,FPZ,AFZ,FZ,FCZ,CZ',
 'mpath': '/Users/smp37/dropbox/projects/moonlight/pops/',
 'no_filter': '0',
 'occiptital': 'O1,IZ,OZ,O2',
 'parietal': 'P1,P3,P5,P7,P9,PZ,P2,P4,P6,P8,P10',
 'parieto-occipital': 'PO7,PO3,POZ,PO8,PO4',
 'posterior': 'P1,P3,P5,P7,P9,PO7,PO3,O1,IZ,OZ,POZ,PZ,P2,P4,P6,P8,P10,PO8,PO4,O2',
 'pre-frontal': 'FP1,FPZ,FP2',
 'right': 'FP2,AF8,AF4,F2,F4,F6,F8,FT8,FC6,FC4,FC2,C2,C4,C6,T8

In [15]:
p = proj.inst( 0 )

___________________________________________________________________
Processing: learn-nsrr01 | tutorial//edfs/learn-nsrr01.edf
 duration 11.22.00, 40920s | time 21.58.17 - 09.20.17 | date 01.01.85

 signals: 14 (of 14) selected in a standard EDF file
  SaO2 | PR | EEG_sec | ECG | EMG | EOG_L | EOG_R | EEG
  AIRFLOW | THOR_RES | ABDO_RES | POSITION | LIGHT | OX_STAT


In [16]:
p.ivars()

{'age': '44',
 'airflow': 'AIRFLOW',
 'ecg': 'ECG',
 'eeg': 'EEG_sec,EEG',
 'effort': 'THOR_RES,ABDO_RES',
 'emg': 'EMG',
 'eog': 'EOG_L,EOG_R',
 'generic': '',
 'hr': 'PR',
 'ic': '',
 'ignore': '',
 'imf': '',
 'leg': '',
 'light': 'LIGHT',
 'oxygen': 'SaO2,OX_STAT',
 'position': 'POSITION',
 'ref': '',
 's1': 'EEG',
 'snore': ''}

In [17]:
proj.eval( 'STATS sig=${s1}' ) 

___________________________________________________________________
Processing: learn-nsrr01 | tutorial//edfs/learn-nsrr01.edf
 duration 11.22.00, 40920s | time 21.58.17 - 09.20.17 | date 01.01.85

 signals: 14 (of 14) selected in a standard EDF file
  SaO2 | PR | EEG_sec | ECG | EMG | EOG_L | EOG_R | EEG
  AIRFLOW | THOR_RES | ABDO_RES | POSITION | LIGHT | OX_STAT
 ..................................................................
 CMD #1: STATS
   options: sig=EEG
 processing EEG ...
___________________________________________________________________
Processing: learn-nsrr02 | tutorial//edfs/learn-nsrr02.edf
 duration 09.57.30, 35850s | time 21.18.06 - 07.15.36 | date 01.01.85

 signals: 14 (of 14) selected in a standard EDF file
  SaO2 | PR | EEG_sec | ECG | EMG | EOG_L | EOG_R | EEG
  AIRFLOW | THOR_RES | ABDO_RES | POSITION | LIGHT | OX_STAT
 ..................................................................
 CMD #1: STATS
   options: sig=EEG_sec
 processing EEG_sec ...
__________

{'STATS: CH':              ID       CH       KURT    MAX      MEAN         MIN         P01  \
 0  learn-nsrr03      ECG  11.165007   1.25  0.003571   -1.240196   -1.240196   
 1  learn-nsrr01      EEG   4.657487  125.0 -0.301199 -124.019608 -124.019608   
 2  learn-nsrr02  EEG_sec    1.94867  125.0  1.544881 -124.019608 -124.019608   
 
           P02         P05        P10  ...       P60        P70       P80  \
 0   -1.240196   -0.259804  -0.142157  ...   0.02451   0.034314  0.063725   
 1 -124.019608  -70.098039 -24.019608  ...   2.45098   4.411765  8.333333   
 2 -124.019608 -124.019608 -40.686275  ...  7.352941  15.196078      25.0   
 
          P90        P95    P98    P99        RMS         SD      SKEW  
 0   0.142157   0.259804   1.25   1.25    0.30003   0.300009 -0.094024  
 1  23.039216  72.058824  125.0  125.0  37.801351  37.800154  0.086212  
 2  44.607843      125.0  125.0  125.0  49.983413  49.959538 -0.047856  
 
 [3 rows x 24 columns]}

In [18]:
lp.resources.MODEL_PATH = '/Users/smp37/dropbox/projects/moonlight/models/'
proj.predict_SUN2019( cen = 'EEG' )

___________________________________________________________________
Processing: learn-nsrr01 | tutorial//edfs/learn-nsrr01.edf
 duration 11.22.00, 40920s | time 21.58.17 - 09.20.17 | date 01.01.85

 signals: 14 (of 14) selected in a standard EDF file
  SaO2 | PR | EEG_sec | ECG | EMG | EOG_L | EOG_R | EEG
  AIRFLOW | THOR_RES | ABDO_RES | POSITION | LIGHT | OX_STAT
 ..................................................................
 CMD #1: FREEZE
   options: ORIG sig=*
  freezing state, with tag ORIG
  copied 40920 records
  currently 1 freeze(s): ORIG
 ..................................................................
 CMD #2: CACHE
   options: cache=p1 record=MTM,MTM,B,CH,STG sig=*
  caching output from MTM, variable = MTM (strata = B,CH,STG) to cache p1
 ..................................................................
 CMD #3: CACHE
   options: cache=p1 record=MTM,SPECKURT,B,STG sig=*
  caching output from MTM, variable = SPECKURT (strata = B,STG) to cache p1
 ...................

{'EPOCH: BL':              ID   DUR GENERIC   INC      NE OFFSET
 0  learn-nsrr01  30.0       0  30.0  1364.0    0.0
 1  learn-nsrr02  30.0       0  30.0  1195.0    0.0
 2  learn-nsrr03  30.0       0  30.0  1364.0    0.0}

ssing EEG ...
 ..................................................................
 CMD #27: SPINDLES
   options: cycles=12 fc=13.5 mag=1.5 max=10 sig=EEG so

 detecting slow waves: 0.5-4Hz
  - relative threshold 1.5x median
  - (based on both P2P and negative peak amplitude)
  - full waves, based on consecutive positive-to-negative zero-crossings
  31766 zero crossings detected
  6362 SWs met criteria (thresholds (<x, >p2p) -0.822057 1.66425)
  running Hilbert transform on filtered signal

  detecting spindles around F_C 13.5Hz for EEG
  wavelet with 12 cycles
  smoothing window = 0.1s
  detection thresholds (core, flank, max)  = 4.5, 2x
  core duration threshold (core, min, max) = 0.3, 0.5, 10s
  basic mean-based multiplicative threshold rule
  merged nearby intervals: from 414 to 406 unique events
  filtering at 11.5 to 15.5, ripple=0.02 tw=4
  QC'ed spindle list from 406 to 363
  estimated spindle frequency is 13.15
  estimated spindle density is 1.38815
 ...........................

In [19]:
proj.strata()

= 4.5, 2x
  core duration threshold (core, min, max) = 0.3, 0.5, 10s
  basic mean-based multiplicative threshold rule
  merged nearby intervals: from 740 to 715 unique events
  filtering at 11.5 to 15.5, ripple=0.02 tw=4
  QC'ed spindle list from 715 to 686
  estimated spindle frequency is 12.8212
  estimated spindle density is 3.4386
 ..................................................................
 CMD #28: THAW
   options: preserve-cache sig=* tag=ALL
  thawing previous freeze ALL
  old dataset   : 11970 records, 15 signals, 10 annotations
  thawed freeze : 35850 records, 14 signals, 10 annotations
  restoring the cache and any recording options
  copied 35850 records
 ..................................................................
 CMD #29: TAG
   options: STG/N3 sig=*
  setting analysis tag to [STG/N3]
 ..................................................................
 CMD #30: MASK
   options: ifnot=N3 sig=*
  set masking mode to 'force'
  annots: N3
  applied annotation ma

,Command,Stata
0,EPOCH,BL
1,MASK,EMASK_STG
2,MTM,B1_B2_CH_STG
3,MTM,B_CH_STG
4,MTM,B_STG
5,MTM,CH_F_STG
6,PREDICT,BL
7,PREDICT,FTR
8,RE,BL
9,RE,STG


  currently 2 freeze(s): ALL ORIG
 ..................................................................
 CMD #2: CACHE
   options: cache=p1 record=MTM,MTM,B,CH,STG sig=*
  caching output from MTM, variable = MTM (strata = B,CH,STG) to cache p1
 ..................................................................
 CMD #3: CACHE
   options: cache=p1 record=MTM,SPECKURT,B,STG sig=*
  caching output from MTM, variable = SPECKURT (strata = B,STG) to cache p1
 ..................................................................
 CMD #4: CACHE
   options: cache=p1 record=MTM,RATIO,B1,B2,CH,STG sig=*
  caching output from MTM, variable = RATIO (strata = B1,B2,CH,STG) to cache p1
 ..................................................................
 CMD #5: CACHE
   options: cache=p1 record=STATS,KURT_MN,CH,STG sig=*
  caching output from STATS, variable = KURT_MN (strata = CH,STG) to cache p1
 ..................................................................
 CMD #6: CACHE
   options: cache=p1 record

In [20]:
proj.table( 'PREDICT' )

,ID,NF,NF_OBS,OKAY,Y,Y1,YOBS
0,learn-nsrr01,13,13,1,48.989031,46.336365,44.0
1,learn-nsrr02,13,13,1,60.597311,63.834363,55.0
2,learn-nsrr03,13,13,1,50.259777,59.386546,66.0


In [21]:
proj.var( 'do_run' , '1' )

In [22]:
proj.vars()

{'anterio-frontal': 'AF7,AF3,AFZ,AF8,AF4',
 'anterior': 'FP1,AF7,AF3,F1,F3,F5,F7,FPZ,AFZ,FZ,FP2,AF8,AF4,F2,F4,F6,F8',
 'cen': 'EEG',
 'central': 'FT7,FC5,FC3,FC1,C1,C3,C5,T7,TP7,CP5,CP3,CP1,CPZ,FCZ,CZ,FT8,FC6,FC4,FC2,C2,C4,C6,T8,TP8,CP6,CP4,CP2',
 'centro-parietal': 'CP5,CP3,CP1,CPZ,CP6,CP4,CP2',
 'do_edger': '1',
 'do_reref': '0',
 'do_run': '1',
 'frontal': 'F1,F3,F5,F7,FZ,F2,F4,F6,F8',
 'fronto-central': 'FC5,FC3,FC1,FCZ,FC6,FC4,FC2',
 'left': 'FP1,AF7,AF3,F1,F3,F5,F7,FT7,FC5,FC3,FC1,C1,C3,C5,T7,TP7,CP5,CP3,CP1,P1,P3,P5,P7,P9,PO7,PO3,O1',
 'lib': 's2',
 'mid-central': 'C1,C3,C5,CZ,C2,C4,C6',
 'midline': 'IZ,OZ,POZ,PZ,CPZ,FPZ,AFZ,FZ,FCZ,CZ',
 'mpath': '/Users/smp37/dropbox/projects/moonlight/models/',
 'no_filter': '0',
 'occiptital': 'O1,IZ,OZ,O2',
 'parietal': 'P1,P3,P5,P7,P9,PZ,P2,P4,P6,P8,P10',
 'parieto-occipital': 'PO7,PO3,POZ,PO8,PO4',
 'posterior': 'P1,P3,P5,P7,P9,PO7,PO3,O1,IZ,OZ,POZ,PZ,P2,P4,P6,P8,P10,PO8,PO4,O2',
 'pre-frontal': 'FP1,FPZ,FP2',
 'right': 'FP2,AF8,AF4,F2,F4,